In [ ]:
from tabulate import tabulate

EXAMPLE_1 = "../example_1.txt"
EXAMPLE_2 = "../example_2.txt"
INPUT = "../input.txt"

In [ ]:
def parse_input(input_file_name):
    map = []
    moves = []
    reading_map = True
    with open(input_file_name, 'r') as f:
        for line in f:
            if line == "\n":
                reading_map = False
            if reading_map:
                map.append([c for c in line.strip().replace("\n", "")])
            else:
                moves.extend([c for c in line.strip().replace("\n", "")])
    return map, moves

In [ ]:
map, moves = parse_input(EXAMPLE_2)
print(tabulate(map))
print(moves)

In [ ]:
def build_obstacle_and_box_map(map):
    start_position = (-1, -1)
    height = len(map)
    width = len(map[0])
    obstacle_map = {"rows": [[]for _ in range(height)], "cols": [[]for _ in range(width)]}
    box_map = {"rows": [[]for _ in range(height)], "cols": [[]for _ in range(width)]}
    for row in range(height):
        for col in range(width):
            if map[row][col] == '#':
                obstacle_map["rows"][row].append(col)
                obstacle_map["cols"][col].append(row)
            elif map[row][col] == 'O':
                box_map["rows"][row].append(col)
                box_map["cols"][col].append(row)
            elif map[row][col] == '@':
                start_position = (row, col)
    return obstacle_map, box_map, start_position

In [ ]:
obstacle_map, box_map, start_position = build_obstacle_and_box_map(map)
print(tabulate(obstacle_map["rows"]), tabulate(obstacle_map["cols"]))
print(tabulate(box_map["rows"]), tabulate(box_map["cols"]))
print(start_position)

First way to move.
Part 2 will require a new way, which could be applied for part 1 as well (it's simpler).
Let's keep the first way here for posterity!
(Checkout the typescript version for the correct way for both parts)

In [ ]:
def move(position, obstacle_map, box_map, direction):
    robot_row, robot_col = position
    match direction:
        case ">":
            # Find first obstacle in the way (the map is enclosed in walls so there's always one)
            obstacle_col = -1
            for col in obstacle_map["rows"][robot_row]:
                if col > robot_col:
                    obstacle_col = col
                    break
            # Find boxes before obstacle
            boxes_cols = []
            for col in box_map["rows"][robot_row]:
                if col <= robot_col:
                    continue
                if col >= obstacle_col:
                    break
                # First box next to robot
                if not boxes_cols and col == robot_col + 1:
                    boxes_cols.append(col)
                    continue
                # Box next to last box
                if (
                    boxes_cols
                    and col == boxes_cols[-1] + 1
                ):
                    boxes_cols.append(col)
                    continue
            if boxes_cols:
                # Check if last box in group is next to obstacle
                if boxes_cols[-1] < obstacle_col - 1:
                    # Remove all boxes that can move
                    for col in boxes_cols:
                        box_map["rows"][robot_row].remove(col)
                        box_map["cols"][col].remove(robot_row)
                    # Put them back
                    nb_of_boxes = len(boxes_cols)
                    for i in range(nb_of_boxes):
                        box_col = robot_col + 2 + i
                        box_map["rows"][robot_row].append(box_col)
                        box_map["cols"][box_col].append(robot_row)
                        box_map["cols"][box_col].sort()
                    # Sort the box map row so that it remains in order
                    box_map["rows"][robot_row].sort()
                    # Update robot position
                    robot_col += 1
            else:
                # Update robot position
                if robot_col + 1 < obstacle_col:
                    robot_col += 1
        case "<":
            # Find first obstacle in the way (the map is enclosed in walls so there's always one)
            obstacle_col = -1
            for col in reversed(obstacle_map["rows"][robot_row]):
                if col < robot_col:
                    obstacle_col = col
                    break
            # Find boxes before obstacle
            boxes_cols = []
            for col in reversed(box_map["rows"][robot_row]):
                if col >= robot_col:
                    continue
                if col <= obstacle_col:
                    break
                # First box next to robot
                if not boxes_cols and col == robot_col - 1:
                    boxes_cols.append(col)
                    continue
                # Box next to last box
                if (
                    boxes_cols
                    and col == boxes_cols[-1] - 1
                ):
                    boxes_cols.append(col)
                    continue
            # Check if last box in group is next to obstacle
            if boxes_cols:
                if boxes_cols[-1] > obstacle_col + 1:
                    # Remove all boxes that can move
                    for col in boxes_cols:
                        box_map["rows"][robot_row].remove(col)
                        box_map["cols"][col].remove(robot_row)
                    # Stack them against each other and put them back
                    nb_of_boxes = len(boxes_cols)
                    for i in range(nb_of_boxes):
                        box_col = robot_col - 2 - i
                        box_map["rows"][robot_row].append(box_col)
                        box_map["cols"][box_col].append(robot_row)
                        box_map["cols"][box_col].sort()
                    # Sort the box map row so that it remains in order
                    box_map["rows"][robot_row].sort()
                    # Update robot position
                    robot_col -= 1
            else:
                # Update robot position
                if robot_col - 1 > obstacle_col:
                    robot_col -= 1
        case "v":
            # Find first obstacle in the way (the map is enclosed in walls so there's always one)
            obstacle_row = -1
            for row in obstacle_map["cols"][robot_col]:
                if row > robot_row:
                    obstacle_row = row
                    break
            # Find boxes before obstacle
            boxes_rows = []
            for row in box_map["cols"][robot_col]:
                if row <= robot_row:
                    continue
                if row >= obstacle_row:
                    break
                # First box next to robot
                if not boxes_rows and row == robot_row + 1:
                    boxes_rows.append(row)
                    continue
                # Box next to last box
                if (
                    boxes_rows
                    and row == boxes_rows[-1] + 1
                ):
                    boxes_rows.append(row)
                    continue
            # Check if last box in group is next to obstacle
            if boxes_rows:
                if boxes_rows[-1] < obstacle_row - 1:
                    # Remove all boxes that can move
                    for row in boxes_rows:
                        box_map["cols"][robot_col].remove(row)
                        box_map["rows"][row].remove(robot_col)
                    # Stack them against each other and put them back
                    nb_of_boxes = len(boxes_rows)
                    for i in range(nb_of_boxes):
                        box_row = robot_row + 2 + i
                        box_map["cols"][robot_col].append(box_row)
                        box_map["rows"][box_row].append(robot_col)
                        box_map["rows"][box_row].sort()
                    # Sort the box map row so that it remains in order
                    box_map["cols"][robot_col].sort()
                    # Update robot position
                    robot_row += 1
            else:
                # Update robot position
                if robot_row + 1 < obstacle_row:
                    robot_row += 1
        case "^":
            # Find first obstacle in the way (the map is enclosed in walls so there's always one)
            obstacle_row = -1
            for row in reversed(obstacle_map["cols"][robot_col]):
                if row < robot_row:
                    obstacle_row = row
                    break
            # Find boxes before obstacle
            boxes_rows = []
            for row in reversed(box_map["cols"][robot_col]):
                if row >= robot_row:
                    continue
                if row <= obstacle_row:
                    break
                # First box next to robot
                if not boxes_rows and row == robot_row - 1:
                    boxes_rows.append(row)
                    continue
                # Box next to last box
                if (
                    boxes_rows
                    and row == boxes_rows[-1] - 1
                ):
                    boxes_rows.append(row)
                    continue
            # Check if last box in group is next to obstacle
            if boxes_rows:
                if boxes_rows[-1] > obstacle_row + 1:
                    # Remove all boxes that can move
                    for row in boxes_rows:
                        box_map["cols"][robot_col].remove(row)
                        box_map["rows"][row].remove(robot_col)
                    # Stack them against each other and put them back
                    nb_of_boxes = len(boxes_rows)
                    for i in range(nb_of_boxes):
                        box_row = robot_row - 2 - i
                        box_map["cols"][robot_col].append(box_row)
                        box_map["rows"][box_row].append(robot_col)
                        box_map["rows"][box_row].sort()
                    # Sort the box map row so that it remains in order
                    box_map["cols"][robot_col].sort()
                    # Update robot position
                    robot_row -= 1
            else:
                # Update robot position
                if robot_row - 1 > obstacle_row:
                    robot_row -= 1
    return (robot_row, robot_col)

In [ ]:
def visualize_map(position, obstacle_map, box_map):
    height = len(obstacle_map["rows"])
    width = len(obstacle_map["cols"])
    grid = [['.' for _ in range(width)] for _ in range(height)]
    for row in range(height):
        for col in range(width):
            if col in obstacle_map["rows"][row]:
                grid[row][col] = '#'
            elif col in box_map["rows"][row]:
                grid[row][col] = 'O'
            elif (row, col) == position:
                grid[row][col] = '@'
    print(tabulate(grid))

In [ ]:
def visualize_run(input_file_name):
    map, moves = parse_input(input_file_name)
    obstacle_map, box_map, start_position = build_obstacle_and_box_map(map)
    position = start_position
    visualize_map(position, obstacle_map, box_map)
    for direction in moves:
        position = move(position, obstacle_map, box_map, direction)
    visualize_map(position, obstacle_map, box_map)

In [ ]:
visualize_run(EXAMPLE_2)

In [ ]:
visualize_run(EXAMPLE_1)

In [ ]:
def gps(box_position):
    box_row, box_col = box_position
    return box_row * 100 + box_col

In [ ]:
def calculate_gps_sum(box_map):
    total = 0
    for row_nb, row in enumerate(box_map["rows"]):
        for col_nb in row:
            total += gps((row_nb, col_nb))
    return total


In [ ]:
def part_1(input_file_name):
    map, moves = parse_input(input_file_name)
    obstacle_map, box_map, start_position = build_obstacle_and_box_map(map)
    position = start_position
    for direction in moves:
        position = move(position, obstacle_map, box_map, direction)
    result = calculate_gps_sum(box_map)
    print(result)

In [ ]:
part_1(EXAMPLE_2)

In [ ]:
part_1(EXAMPLE_1)

In [ ]:
part_1(INPUT)

In [ ]:
map, moves = parse_input(EXAMPLE_1)
print(tabulate(map))
print(moves)

In [ ]:
def build_obstacle_and_box_map(map):
    start_position = (-1, -1)
    height = len(map)
    width = len(map[0])
    obstacle_map = {"rows": [[]for _ in range(height)], "cols": [[]for _ in range(width*2)]}
    box_map = {"rows": [[]for _ in range(height)], "cols": [[]for _ in range(width*2)]}
    for row in range(height):
        for col in range(width):
            if map[row][col] == '#':
                obstacle_map["rows"][row].append(col*2)
                obstacle_map["rows"][row].append(col*2+1)
                obstacle_map["cols"][col*2].append(row)
                obstacle_map["cols"][col*2+1].append(row)
            elif map[row][col] == 'O':
                box_map["rows"][row].append(col*2)
                box_map["cols"][col*2].append(row)
                box_map["cols"][col*2+1].append(row)
            elif map[row][col] == '@':
                start_position = (row, col*2)
    return obstacle_map, box_map, start_position

In [ ]:
obstacle_map, box_map, start_position = build_obstacle_and_box_map(map)
print(tabulate(obstacle_map["rows"]), tabulate(obstacle_map["cols"]))
print(tabulate(box_map["rows"]), tabulate(box_map["cols"]))
print(start_position)

In [ ]:
def visualize_map(position, obstacle_map, box_map):
    height = len(obstacle_map["rows"])
    width = len(obstacle_map["cols"])
    grid = [['.' for _ in range(width)] for _ in range(height)]
    for row in range(height):
        for col in range(width):
            if col in obstacle_map["rows"][row]:
                grid[row][col] = '#'
            elif col in box_map["rows"][row]:
                grid[row][col] = '['
                grid[row][col+1] = ']'
            elif (row, col) == position:
                grid[row][col] = '@'
    print(tabulate(grid))

In [ ]:
visualize_map(start_position, obstacle_map, box_map)

In [ ]:
def move(position, obstacle_map, box_map, direction):
    robot_row, robot_col = position
    match direction:
        case ">":
            current_col = robot_col+1
            boxes_to_move = set()
            movable = True
            # Let's go right one column at a time and check that there's no obstacle blocking the way
            while movable:
                if robot_row in obstacle_map["cols"][current_col]:
                    # An obstacle is blocking the way
                    movable = False
                    break
                if robot_row in box_map["cols"][current_col]:
                    # The left side of a box is in the way
                    boxes_to_move.add((robot_row, current_col))
                    # Boxes are 2 spaces wide, we'll need to check two spaces right from this box
                    current_col += 2
                else:
                    # Nothing is in the way, we can move
                    break
            if not movable:
                # An obstacle blocked us
                return (robot_row, robot_col)
            # We were never blocked and we've reached a column with no boxes in the robot's row
            # So we know we can move, so let's move all the boxes and the robot
            for (row, col) in boxes_to_move:
                # Remove the box
                box_map["cols"][col].remove(row)
                box_map["cols"][col+1].remove(row)
                box_map["rows"][row].remove(col)
                # Put it back
                box_map["cols"][col+1].append(row)
                box_map["cols"][col+2].append(row)
                box_map["rows"][row].append(col+1)
                box_map["cols"][col+1].sort()
                box_map["cols"][col+2].sort()
                box_map["rows"][row].sort()
            robot_col += 1
        case "<":
            current_col = robot_col-1
            boxes_to_move = set()
            movable = True
            # Let's go left one column at a time and check that there's no obstacle blocking the way
            while movable:
                if robot_row in obstacle_map["cols"][current_col]:
                    # An obstacle is blocking the way
                    movable = False
                    break
                if robot_row in box_map["cols"][current_col]:
                    # The right side of a box is in the way (the col coordinate of a box is always its left side)
                    boxes_to_move.add((robot_row, current_col-1))
                    # Boxes are 2 spaces wide, we'll need to check two spaces left from this box
                    current_col -= 2
                else:
                    break
            if not movable:
                # An obstacle blocked us
                return (robot_row, robot_col)
            # We were never blocked and we've reached a column with no boxes in the robot's row
            # So we know we can move, so let's move all the boxes and the robot
            for (row, col) in boxes_to_move:
                # Remove the box
                box_map["cols"][col].remove(row)
                box_map["cols"][col+1].remove(row)
                box_map["rows"][row].remove(col)
                # Put it back
                box_map["cols"][col-1].append(row)
                box_map["cols"][col].append(row)
                box_map["rows"][row].append(col-1)
                box_map["cols"][col-1].sort()
                box_map["cols"][col].sort()
                box_map["rows"][row].sort()
            robot_col -= 1
        case "v":
            current_row = robot_row+1
            boxes_to_move = set()
            cols_to_check = set([robot_col])
            movable = True
            # Let's go down one row at a time and check that there's no obstacle blocking the way
            # For each row we need to check a number of columns for obstacles
            while movable and cols_to_check:
                new_cols_to_check = set()
                for col in cols_to_check:
                    if col in obstacle_map["rows"][current_row]:
                        # An obstacle is blocking the way
                        movable = False
                        break
                    if col in box_map["rows"][current_row]:
                        # The left side of a box is in the way
                        boxes_to_move.add((current_row, col))
                        new_cols_to_check.add(col)
                        new_cols_to_check.add(col+1)
                        continue
                    if col-1 in box_map["rows"][current_row]:
                        # The right side of a box is in the way
                        boxes_to_move.add((current_row, col-1))
                        new_cols_to_check.add(col-1)
                        new_cols_to_check.add(col)
                        continue
                if not movable:
                    break
                current_row += 1
                cols_to_check = new_cols_to_check
            if not movable:
                # An obstacle blocked us at some point
                return (robot_row, robot_col)
            # We were never blocked and we've reached a row with no boxes in the columns we needed to check
            # So we know we can move, so let's move all the boxes and the robot
            for (row, col) in boxes_to_move:
                # Remove the boxe
                box_map["cols"][col].remove(row)
                box_map["cols"][col+1].remove(row)
                box_map["rows"][row].remove(col)
                # Put it back
                box_map["cols"][col].append(row+1)
                box_map["cols"][col+1].append(row+1)
                box_map["rows"][row+1].append(col)
                box_map["cols"][col].sort()
                box_map["cols"][col+1].sort()
                box_map["rows"][row+1].sort()
            robot_row += 1
        case "^":
            current_row = robot_row-1
            boxes_to_move = set()
            cols_to_check = set([robot_col])
            movable = True
            # Let's go up one row at a time and check that there's no obstacle blocking the way
            # For each row we need to check a number of columns for obstacles
            while movable and cols_to_check:
                new_cols_to_check = set()
                for col in cols_to_check:
                    if col in obstacle_map["rows"][current_row]:
                        # An obstacle is blocking the way
                        movable = False
                        break
                    if col in box_map["rows"][current_row]:
                        # The left side of a box is in the way
                        boxes_to_move.add((current_row, col))
                        new_cols_to_check.add(col)
                        new_cols_to_check.add(col+1)
                        continue
                    if col-1 in box_map["rows"][current_row]:
                        # The right side of a box is in the way
                        boxes_to_move.add((current_row, col-1))
                        new_cols_to_check.add(col-1)
                        new_cols_to_check.add(col)
                        continue
                if not movable:
                    break
                current_row -= 1
                cols_to_check = new_cols_to_check
            if not movable:
                # An obstacle blocked us at some point
                return (robot_row, robot_col)
            # We were never blocked and we've reached a row with no boxes in the columns we needed to check
            # So we know we can move, so let's move all the boxes and the robot
            for (row, col) in boxes_to_move:
                # Remove the box
                box_map["cols"][col].remove(row)
                box_map["cols"][col+1].remove(row)
                box_map["rows"][row].remove(col)
                # Put it back
                box_map["cols"][col].append(row-1)
                box_map["cols"][col+1].append(row-1)
                box_map["rows"][row-1].append(col)
                box_map["cols"][col].sort()
                box_map["cols"][col+1].sort()
                box_map["rows"][row-1].sort()
            robot_row -= 1
    return (robot_row, robot_col)

In [ ]:
position = start_position
visualize_map(position, obstacle_map, box_map)
for direction in moves:
    print(position, direction)
    position = move(position, obstacle_map, box_map, direction)
    visualize_map(position, obstacle_map, box_map)

In [ ]:
def visualize_run(input_file_name):
    map, moves = parse_input(input_file_name)
    obstacle_map, box_map, start_position = build_obstacle_and_box_map(map)
    position = start_position
    visualize_map(position, obstacle_map, box_map)
    for direction in moves:
        position = move(position, obstacle_map, box_map, direction)
    visualize_map(position, obstacle_map, box_map)

In [ ]:
visualize_run(EXAMPLE_1)

In [ ]:
def part_2(input_file_name):
    map, moves = parse_input(input_file_name)
    obstacle_map, box_map, start_position = build_obstacle_and_box_map(map)
    position = start_position
    for direction in moves:
        position = move(position, obstacle_map, box_map, direction)
    result = calculate_gps_sum(box_map)
    print(result)

In [ ]:
part_2(EXAMPLE_1)

In [ ]:
part_2(INPUT)